In [21]:
import pandas as pd
from thefuzz import fuzz, process

In [22]:
# Đọc danh sách các country theo tên chuẩn lấy từ GeoNames
countries = pd.read_csv('../data/country.csv')["Country"]

In [23]:
# Đọc các tập dữ liệu
co2_df = pd.read_csv('../data/co2.csv').rename(columns={"value": "co2"})
climate_change_df = pd.read_csv('../data/climate_change.csv').rename(columns={"value": "climate_change"})
energy_df = pd.read_csv('../data/energy.csv').rename(columns={"value": "energy"})

In [24]:
def find_best_match(name):
  best_match = process.extractOne(name, countries, scorer=fuzz.WRatio)
  return best_match[0], best_match[1]

In [25]:
co2_df_country = co2_df.loc[:, ["country"]].drop_duplicates(subset="country")

In [26]:
matches = co2_df_country["country"].apply(find_best_match)
co2_df_country[["normalized_country", 'match_score']] = pd.DataFrame(matches.to_list(), index=co2_df_country.index)

In [27]:
co2_df_country[co2_df_country["match_score"] < 100].sort_values(by="match_score")
# kiểm tra match_score truớc
# bỏ global, international
# với mấy cái điểm thấp thì mapping bằng tay, ví dụ USA -> United State

,country,normalized_country,match_score
60554,Global,Gibraltar,60
12604,Côte d'Ivoire,Israel,60
54800,Türkiye,Turkey,67
60006,International Shipping,Tonga,72
57266,USA,Saint Barthelemy,72
60280,International Aviation,Latvia,75
9590,Cape Verde,Cabo Verde,80
45210,Ryukyu Islands,South Georgia and the South Sandwich Islands,86
39182,State of Palestine,Democratic Republic of the Congo,86
28222,Kuwaiti Oil Fires,Kuwait,90


In [30]:
# Chuẩn hóa lại các quốc gia hàm chuẩn hóa chưa thực hiện được
def normalize_country_name(df, column_name):

    country_mapping = {
        "USA": "United States",
        "Côte d'Ivoire": "Ivory Coast",
        "Ryukyu Islands": "Japan",
        "State of Palestine": "Palestinian Territory"
    }
    # Thay thế tên quốc gia dựa trên từ điển
    df[column_name] = df[column_name].replace(country_mapping)
    return df

# Áp dụng hàm 
co2_df = normalize_country_name(co2_df, "country")
energy_df = normalize_country_name(energy_df, "country")
climate_change_df = normalize_country_name(climate_change_df, "country")

In [ ]:
# Loại bỏ Global/International
def remove_global_entries(df, column_name):
    # Lọc các hàng không có giá trị 'Global' và không chứa 'International'
    return df[~df[column_name].str.contains("International", na=False) & (df[column_name] != "Global")]

# Áp dụng hàm
co2_df = remove_global_entries(co2_df, "country")
energy_df = remove_global_entries(energy_df, "country")
climate_change_df = remove_global_entries(climate_change_df, "country")
